# 画像変換テスト

このノートブックでは、以下の画像変換を段階的に適用し、各変換の前後を比較表示します：

1. **明るさ・コントラスト調整**
   - ガンマ補正 (Gamma Correction)
   - 線形スケーリング (Brightness/Contrast)

2. **ヒストグラム均等化／CLAHE**
   - グローバル均等化
   - CLAHE (適応的ヒストグラム均等化)

3. **ノイズ除去・輪郭強調**
   - 非局所平均フィルタ
   - アンシャープマスク（シャープ化）

4. **クロップ＆リサイズ**
   - 中心部分のクロップ
   - 推奨解像度へのリサイズ

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path

# 日本語フォントの設定
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['figure.figsize'] = (15, 10)

# 表示用関数
def show_image_comparison(original, processed, title, subtitle=""):
    """2つの画像を横並びで比較表示"""
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # 元画像
    axes[0].imshow(original, cmap='gray')
    axes[0].set_title('Original')
    axes[0].axis('off')
    
    # 処理後画像
    axes[1].imshow(processed, cmap='gray')
    axes[1].set_title('Processed')
    axes[1].axis('off')
    
    plt.suptitle(f"{title}\n{subtitle}", fontsize=14)
    plt.tight_layout()
    plt.show()

def show_histogram_comparison(original, processed, title):
    """ヒストグラムを比較表示"""
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    
    # 元画像とヒストグラム
    axes[0, 0].imshow(original, cmap='gray')
    axes[0, 0].set_title('Original Image')
    axes[0, 0].axis('off')
    
    axes[0, 1].hist(original.ravel(), bins=256, range=[0, 256], color='blue', alpha=0.7)
    axes[0, 1].set_title('Original Histogram')
    axes[0, 1].set_xlabel('Pixel Value')
    axes[0, 1].set_ylabel('Frequency')
    
    # 処理後画像とヒストグラム
    axes[1, 0].imshow(processed, cmap='gray')
    axes[1, 0].set_title('Processed Image')
    axes[1, 0].axis('off')
    
    axes[1, 1].hist(processed.ravel(), bins=256, range=[0, 256], color='red', alpha=0.7)
    axes[1, 1].set_title('Processed Histogram')
    axes[1, 1].set_xlabel('Pixel Value')
    axes[1, 1].set_ylabel('Frequency')
    
    plt.suptitle(title, fontsize=14)
    plt.tight_layout()
    plt.show()

print("ライブラリとユーティリティ関数の準備完了！")

In [ ]:
# 画像パスの設定（テスト用画像を指定）
# 例: OK画像の1つを使用
image_path = r"input\背景カットなし\sample\OK\OK3.png"

# 日本語パス対応のための画像読み込み関数
def read_image_japanese_path(image_path, color_flag=cv2.IMREAD_GRAYSCALE):
    """日本語パスに対応した画像読み込み関数"""
    try:
        # NumPyを使用してバイナリファイルを読み込み
        with open(image_path, 'rb') as f:
            binary_data = f.read()
        
        # バイナリデータからNumPy配列に変換
        img_array = np.frombuffer(binary_data, dtype=np.uint8)
        
        # OpenCVでデコード
        img = cv2.imdecode(img_array, color_flag)
        
        return img
    except Exception as e:
        print(f"画像読み込みエラー: {e}")
        return None

# 画像が存在するかチェック
if os.path.exists(image_path):
    print(f"画像パス: {image_path}")
    print("画像が見つかりました！")
    
    # 画像読み込み（グレースケール）- 日本語パス対応
    original_img = read_image_japanese_path(image_path, cv2.IMREAD_GRAYSCALE)
    
    if original_img is not None:
        print(f"画像サイズ: {original_img.shape}")
        
        # 元画像を表示
        plt.figure(figsize=(8, 6))
        plt.imshow(original_img, cmap='gray')
        plt.title(f"Original Image: {os.path.basename(image_path)}")
        plt.axis('off')
        plt.show()
        
        # 基本統計情報
        print(f"最小値: {original_img.min()}")
        print(f"最大値: {original_img.max()}")
        print(f"平均値: {original_img.mean():.2f}")
        print(f"標準偏差: {original_img.std():.2f}")
    else:
        print("画像の読み込みに失敗しました")
        print("ファイル形式またはパスに問題がある可能性があります")
else:
    print(f"画像が見つかりません: {image_path}")
    print("パスを確認してください")
    
    # 代替画像パスを提案
    print("\n=== 利用可能な画像ファイル ===")
    try:
        sample_dir = r"input\背景カットなし\sample"
        for category in os.listdir(sample_dir):
            category_path = os.path.join(sample_dir, category)
            if os.path.isdir(category_path):
                print(f"\n{category}:")
                for file in os.listdir(category_path):
                    if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                        print(f"  - {os.path.join(category_path, file)}")
    except Exception as e:
        print(f"ディレクトリ読み込みエラー: {e}")

## 1. 明るさ・コントラスト調整

### 1-1. ガンマ補正 (Gamma Correction)
ガンマ補正は、画像の明るさを非線形的に調整する手法です。
- γ < 1: 暗い部分を明るくする（全体的に明るくなる）
- γ > 1: 明るい部分を暗くする（全体的に暗くなる）

In [ ]:
# ガンマ補正の実装
def gamma_correction(image, gamma):
    """ガンマ補正を適用"""
    inv_gamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv_gamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image, table)

# 異なるガンマ値でテスト
gamma_values = [0.5, 1.0, 1.5, 2.0]

# 現在の画像をコピー
current_img = original_img.copy()

print("ガンマ補正の比較:")
for gamma in gamma_values:
    img_gamma = gamma_correction(current_img, gamma)
    
    # 比較表示
    show_image_comparison(
        current_img, 
        img_gamma, 
        f"Gamma Correction (γ = {gamma})",
        f"暗部補正: {'強' if gamma < 1 else '弱' if gamma > 1 else 'なし'}"
    )
    
    # ヒストグラム比較
    show_histogram_comparison(
        current_img,
        img_gamma,
        f"Histogram Comparison - Gamma = {gamma}"
    )

# 最適なガンマ値を選択（例：1.5）
optimal_gamma = 1.5
img_gamma = gamma_correction(current_img, optimal_gamma)

print(f"\n最適なガンマ値 {optimal_gamma} を適用しました")

### 1-2. 線形スケーリング (Brightness/Contrast)
線形変換： output = α × input + β
- α (alpha): コントラスト係数（1.0 = 変化なし、>1.0 = 高コントラスト）
- β (beta): 明るさオフセット（0 = 変化なし、>0 = 明るく、<0 = 暗く）

In [ ]:
# 線形スケーリングのテスト
def linear_scaling(image, alpha, beta):
    """線形スケーリングを適用"""
    return cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

# 異なるパラメータでテスト
alpha_values = [0.8, 1.0, 1.2, 1.5]  # コントラスト
beta_values = [-20, 0, 20, 40]        # 明るさ

print("線形スケーリングの比較:")

# コントラストの効果を確認
current_img = img_gamma.copy()
for alpha in alpha_values:
    img_contrast = linear_scaling(current_img, alpha, 0)
    
    show_image_comparison(
        current_img, 
        img_contrast, 
        f"Contrast Adjustment (α = {alpha})",
        f"コントラスト: {'低' if alpha < 1 else '高' if alpha > 1 else '標準'}"
    )

# 明るさの効果を確認
for beta in beta_values:
    img_brightness = linear_scaling(current_img, 1.0, beta)
    
    show_image_comparison(
        current_img, 
        img_brightness, 
        f"Brightness Adjustment (β = {beta})",
        f"明るさ: {'暗' if beta < 0 else '明' if beta > 0 else '標準'}"
    )

# 最適な値を組み合わせて適用
optimal_alpha = 1.2  # コントラスト
optimal_beta = 20    # 明るさ

img_bc = linear_scaling(current_img, optimal_alpha, optimal_beta)

# 最終結果を表示
show_image_comparison(
    current_img, 
    img_bc, 
    f"Final Brightness & Contrast (α={optimal_alpha}, β={optimal_beta})",
    "ガンマ補正 + 線形スケーリング"
)

print(f"\n最適なパラメータ (α={optimal_alpha}, β={optimal_beta}) を適用しました")

## 2. ヒストグラム均等化／CLAHE

### 2-1. グローバルヒストグラム均等化
画像全体のヒストグラムを均等化し、コントラストを改善します。

In [ ]:
# グローバルヒストグラム均等化
current_img = original_img.copy()

# ヒストグラム均等化を適用
hist_eq = cv2.equalizeHist(current_img)

# 比較表示
show_image_comparison(
    current_img, 
    hist_eq, 
    "Global Histogram Equalization",
    "グローバルなコントラスト改善"
)

# ヒストグラム比較
show_histogram_comparison(
    current_img,
    hist_eq,
    "Global Histogram Equalization - Histogram Comparison"
)

print("グローバルヒストグラム均等化を適用しました")

### 2-2. CLAHE (Contrast Limited Adaptive Histogram Equalization)
適応的ヒストグラム均等化により、局所的なコントラストを改善しつつ、過度な増強を抑制します。

In [ ]:
# CLAHE (適応的ヒストグラム均等化)
# 異なるパラメータでテスト
clip_limits = [1.0, 2.0, 3.0, 4.0]
tile_sizes = [(4, 4), (8, 8), (16, 16)]

print("CLAHE パラメータの比較:")

# Clip Limitの効果を確認
current_img = original_img.copy()
for clip_limit in clip_limits:
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=(8, 8))
    clahe_eq = clahe.apply(current_img)
    
    show_image_comparison(
        current_img, 
        clahe_eq, 
        f"CLAHE (clipLimit={clip_limit})",
        f"制限値: {'低' if clip_limit < 2 else '高' if clip_limit > 2 else '標準'}"
    )

# Tile Grid Sizeの効果を確認
for tile_size in tile_sizes:
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=tile_size)
    clahe_eq = clahe.apply(current_img)
    
    show_image_comparison(
        current_img, 
        clahe_eq, 
        f"CLAHE (tileGridSize={tile_size})",
        f"タイルサイズ: {'小' if tile_size[0] < 8 else '大' if tile_size[0] > 8 else '標準'}"
    )

# 最適なパラメータを選択
optimal_clip_limit = 2.0
optimal_tile_size = (8, 8)

clahe = cv2.createCLAHE(clipLimit=optimal_clip_limit, tileGridSize=optimal_tile_size)
clahe_eq = clahe.apply(current_img)

# CLAHE vs グローバル均等化の比較
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(current_img, cmap='gray')
axes[0].set_title('Original')
axes[0].axis('off')

axes[1].imshow(hist_eq, cmap='gray')
axes[1].set_title('Global Histogram Equalization')
axes[1].axis('off')

axes[2].imshow(clahe_eq, cmap='gray')
axes[2].set_title('CLAHE')
axes[2].axis('off')

plt.suptitle("Histogram Equalization Methods Comparison", fontsize=14)
plt.tight_layout()
plt.show()

print(f"\n最適なCLAHEパラメータ (clipLimit={optimal_clip_limit}, tileGridSize={optimal_tile_size}) を適用しました")

## 3. ノイズ除去・輪郭強調

### 3-1. 非局所平均フィルタ (Non-Local Means Denoising)
画像のディテールを保持しながらノイズを除去します。

In [ ]:
# ノイズ除去フィルタのテスト
current_img = original_img.copy()

# 異なるパラメータでテスト
h_values = [10, 30, 50]  # ノイズ除去の強さ
template_window_sizes = [5, 7, 9]  # テンプレートウィンドウサイズ

print("ノイズ除去フィルタの比較:")

# h値の効果を確認
for h in h_values:
    denoised = cv2.fastNlMeansDenoising(current_img, None, h=h, templateWindowSize=7, searchWindowSize=21)
    
    show_image_comparison(
        current_img, 
        denoised, 
        f"Non-Local Means Denoising (h={h})",
        f"ノイズ除去強度: {'弱' if h < 30 else '強' if h > 30 else '標準'}"
    )

# テンプレートウィンドウサイズの効果を確認
for template_size in template_window_sizes:
    denoised = cv2.fastNlMeansDenoising(current_img, None, h=30, templateWindowSize=template_size, searchWindowSize=21)
    
    show_image_comparison(
        current_img, 
        denoised, 
        f"Non-Local Means Denoising (templateWindowSize={template_size})",
        f"テンプレートサイズ: {'小' if template_size < 7 else '大' if template_size > 7 else '標準'}"
    )

# 最適なパラメータを選択
optimal_h = 30
optimal_template_size = 7
optimal_search_size = 21

denoised = cv2.fastNlMeansDenoising(
    current_img, 
    None, 
    h=optimal_h, 
    templateWindowSize=optimal_template_size, 
    searchWindowSize=optimal_search_size
)

print(f"\n最適なノイズ除去パラメータ (h={optimal_h}, templateWindowSize={optimal_template_size}) を適用しました")

### 3-2. アンシャープマスク (Sharpening)
エッジを強調してディテールを鮮明にします。

In [ ]:
# アンシャープマスク（シャープ化）
current_img = original_img.copy()

# 異なるシャープ化カーネルをテスト
kernels = {
    "Mild Sharpening": np.array([[0, -1, 0],
                                [-1, 5, -1],
                                [0, -1, 0]]),
    
    "Standard Sharpening": np.array([[-1, -1, -1],
                                    [-1, 9, -1],
                                    [-1, -1, -1]]),
    
    "Strong Sharpening": np.array([[0, -1, 0],
                                  [-1, 6, -1],
                                  [0, -1, 0]]),
    
    "Edge Enhancement": np.array([[-1, -1, -1],
                                 [-1, 8, -1],
                                 [-1, -1, -1]]) / 8
}

print("シャープ化カーネルの比較:")

# 各カーネルの効果を確認
sharpened_results = {}
for kernel_name, kernel in kernels.items():
    sharpened = cv2.filter2D(current_img, -1, kernel)
    sharpened_results[kernel_name] = sharpened
    
    show_image_comparison(
        current_img, 
        sharpened, 
        f"Sharpening: {kernel_name}",
        f"エッジ強調度: {kernel_name.split()[0]}"
    )

# 最適なカーネルを選択
optimal_kernel_name = "Mild Sharpening"
optimal_kernel = kernels[optimal_kernel_name]

sharpened = cv2.filter2D(current_img, -1, optimal_kernel)

# ノイズ除去 → シャープ化の効果を確認
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(clahe_eq, cmap='gray')
axes[0].set_title('Before Denoising')
axes[0].axis('off')

axes[1].imshow(denoised, cmap='gray')
axes[1].set_title('After Denoising')
axes[1].axis('off')

axes[2].imshow(sharpened, cmap='gray')
axes[2].set_title('After Sharpening')
axes[2].axis('off')

plt.suptitle("Denoising → Sharpening Process", fontsize=14)
plt.tight_layout()
plt.show()

print(f"\n最適なシャープ化カーネル ({optimal_kernel_name}) を適用しました")

## 4. クロップ＆リサイズ

### 4-1. 中心クロップ
画像の中心部分を取り出して、背景ノイズを除去します。

In [ ]:
# クロップ＆リサイズ
current_img = sharpened.copy()

def center_crop(image, crop_size):
    """画像の中心部分をクロップ"""
    h, w = image.shape
    cx, cy = w // 2, h // 2
    
    # クロップサイズを画像サイズに合わせて調整
    crop_h, crop_w = crop_size
    crop_h = min(crop_h, h)
    crop_w = min(crop_w, w)
    
    # クロップ領域を計算
    y1 = max(0, cy - crop_h // 2)
    y2 = min(h, cy + crop_h // 2)
    x1 = max(0, cx - crop_w // 2)
    x2 = min(w, cx + crop_w // 2)
    
    return image[y1:y2, x1:x2]

# 元画像のサイズを確認
h, w = current_img.shape
print(f"元画像サイズ: {w} x {h}")

# 異なるクロップサイズをテスト
crop_sizes = [
    (min(h, w) // 2, min(h, w) // 2),  # 小さなクロップ
    (min(h, w) * 3 // 4, min(h, w) * 3 // 4),  # 中程度のクロップ
    (min(h, w), min(h, w))  # 正方形クロップ
]

print("クロップサイズの比較:")

cropped_results = {}
for i, crop_size in enumerate(crop_sizes):
    cropped = center_crop(current_img, crop_size)
    cropped_results[f"Crop_{i+1}"] = cropped
    
    show_image_comparison(
        current_img, 
        cropped, 
        f"Center Crop ({crop_size[0]} x {crop_size[1]})",
        f"クロップサイズ: {crop_size[0]} x {crop_size[1]}"
    )

# 最適なクロップサイズを選択
optimal_crop_size = crop_sizes[1]  # 中程度のクロップ
cropped = center_crop(current_img, optimal_crop_size)

print(f"\n最適なクロップサイズ ({optimal_crop_size[0]} x {optimal_crop_size[1]}) を適用しました")

### 4-2. リサイズ
モデルの推奨解像度に合わせてリサイズします。

In [ ]:
# リサイズ
current_img = cropped.copy()

# 異なるリサイズサイズをテスト
resize_sizes = [
    (256, 256),
    (512, 512),
    (768, 768),
    (1024, 1024)
]

# 異なる補間方法をテスト
interpolation_methods = {
    "INTER_NEAREST": cv2.INTER_NEAREST,
    "INTER_LINEAR": cv2.INTER_LINEAR,
    "INTER_CUBIC": cv2.INTER_CUBIC,
    "INTER_AREA": cv2.INTER_AREA,
    "INTER_LANCZOS4": cv2.INTER_LANCZOS4
}

print("リサイズサイズの比較:")

# 各サイズの効果を確認
for size in resize_sizes:
    resized = cv2.resize(current_img, size, interpolation=cv2.INTER_AREA)
    
    show_image_comparison(
        current_img, 
        resized, 
        f"Resize to {size[0]} x {size[1]}",
        f"解像度: {size[0]} x {size[1]}"
    )

print("\n補間方法の比較:")

# 補間方法の効果を確認（512x512でテスト）
target_size = (512, 512)
for method_name, method in interpolation_methods.items():
    resized = cv2.resize(current_img, target_size, interpolation=method)
    
    show_image_comparison(
        current_img, 
        resized, 
        f"Resize with {method_name}",
        f"補間方法: {method_name}"
    )

# 最適なパラメータを選択
optimal_size = (512, 512)
optimal_interpolation = cv2.INTER_AREA

final_image = cv2.resize(current_img, optimal_size, interpolation=optimal_interpolation)

print(f"\n最適なリサイズパラメータ (size={optimal_size}, interpolation=INTER_AREA) を適用しました")

## 5. 最終結果の比較

すべての変換処理を適用した結果を、元画像と比較して確認します。

In [ ]:
# 最終結果の比較表示
def show_preprocessing_pipeline(original, steps_images, step_names, title="Image Preprocessing Pipeline"):
    """前処理パイプラインの全段階を表示"""
    num_steps = len(steps_images)
    cols = min(4, num_steps + 1)  # 最大4列
    rows = (num_steps + 1 + cols - 1) // cols  # 必要な行数
    
    fig, axes = plt.subplots(rows, cols, figsize=(20, 5 * rows))
    axes = axes.flatten() if rows > 1 else [axes] if cols == 1 else axes
    
    # 元画像
    axes[0].imshow(original, cmap='gray')
    axes[0].set_title('Original Image')
    axes[0].axis('off')
    
    # 各段階の結果
    for i, (img, name) in enumerate(zip(steps_images, step_names)):
        axes[i + 1].imshow(img, cmap='gray')
        axes[i + 1].set_title(name)
        axes[i + 1].axis('off')
    
    # 使用しない軸を非表示
    for i in range(num_steps + 1, len(axes)):
        axes[i].axis('off')
    
    plt.suptitle(title, fontsize=16)
    plt.tight_layout()
    plt.show()

# 処理段階の画像を収集
preprocessing_steps = [
    img_gamma,    # ガンマ補正
    img_bc,       # 線形スケーリング
    clahe_eq,     # CLAHE
    denoised,     # ノイズ除去
    sharpened,    # シャープ化
    cropped,      # クロップ
    final_image   # 最終リサイズ
]

step_names = [
    "1. Gamma Correction",
    "2. Brightness & Contrast",
    "3. CLAHE",
    "4. Denoising",
    "5. Sharpening",
    "6. Cropping",
    "7. Final Resize"
]

# パイプライン全体の表示
show_preprocessing_pipeline(
    original_img, 
    preprocessing_steps, 
    step_names,
    "Complete Image Preprocessing Pipeline"
)

# 元画像と最終結果の比較
fig, axes = plt.subplots(1, 2, figsize=(15, 7))

axes[0].imshow(original_img, cmap='gray')
axes[0].set_title(f'Original Image\nSize: {original_img.shape}')
axes[0].axis('off')

axes[1].imshow(final_image, cmap='gray')
axes[1].set_title(f'Processed Image\nSize: {final_image.shape}')
axes[1].axis('off')

plt.suptitle("Before & After Preprocessing", fontsize=16)
plt.tight_layout()
plt.show()

# 処理前後の統計情報比較
print("=== 処理前後の統計情報比較 ===")
print(f"元画像 - サイズ: {original_img.shape}, 平均: {original_img.mean():.2f}, 標準偏差: {original_img.std():.2f}")
print(f"最終画像 - サイズ: {final_image.shape}, 平均: {final_image.mean():.2f}, 標準偏差: {final_image.std():.2f}")

# 最終画像を保存
output_path = "preprocessed_image.png"
cv2.imwrite(output_path, final_image)
print(f"\n処理済み画像を保存しました: {output_path}")

print("\n=== 処理パイプライン完了 ===")
print("1. ガンマ補正 → 暗部を明るく調整")
print("2. 線形スケーリング → コントラストと明るさを調整")
print("3. CLAHE → 適応的ヒストグラム均等化")
print("4. ノイズ除去 → 非局所平均フィルタでノイズを除去")
print("5. シャープ化 → エッジを強調")
print("6. クロップ → 中心部分を抽出")
print("7. リサイズ → モデル推奨解像度に調整")

## 6. パラメータ調整用（オプション）

このセクションでは、異なる画像に対してパラメータを簡単に調整できるようにします。

In [ ]:
# パラメータ調整用関数
def preprocess_image_with_params(image_path, 
                                gamma=1.5, 
                                alpha=1.2, 
                                beta=20,
                                clip_limit=2.0,
                                tile_size=(8, 8),
                                denoise_h=30,
                                template_size=7,
                                sharpen_strength="mild",
                                crop_ratio=0.75,
                                final_size=(512, 512)):
    """
    指定されたパラメータで画像を前処理する関数
    
    Parameters:
    - image_path: 画像のパス
    - gamma: ガンマ補正値
    - alpha: コントラスト係数
    - beta: 明るさオフセット
    - clip_limit: CLAHEのクリップ制限
    - tile_size: CLAHEのタイルサイズ
    - denoise_h: ノイズ除去の強さ
    - template_size: ノイズ除去のテンプレートサイズ
    - sharpen_strength: シャープ化の強さ ("mild", "standard", "strong")
    - crop_ratio: クロップ比率
    - final_size: 最終リサイズサイズ
    """
    
    # 画像読み込み（日本語パス対応）
    img = read_image_japanese_path(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f"画像の読み込みに失敗: {image_path}")
        return None
    
    # 1. ガンマ補正
    inv_gamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv_gamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
    img = cv2.LUT(img, table)
    
    # 2. 線形スケーリング
    img = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
    
    # 3. CLAHE
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_size)
    img = clahe.apply(img)
    
    # 4. ノイズ除去
    img = cv2.fastNlMeansDenoising(img, None, h=denoise_h, templateWindowSize=template_size, searchWindowSize=21)
    
    # 5. シャープ化
    sharpen_kernels = {
        "mild": np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]]),
        "standard": np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]]),
        "strong": np.array([[0, -1, 0], [-1, 6, -1], [0, -1, 0]])
    }
    kernel = sharpen_kernels.get(sharpen_strength, sharpen_kernels["mild"])
    img = cv2.filter2D(img, -1, kernel)
    
    # 6. クロップ
    h, w = img.shape
    crop_size = int(min(h, w) * crop_ratio)
    img = center_crop(img, (crop_size, crop_size))
    
    # 7. リサイズ
    img = cv2.resize(img, final_size, interpolation=cv2.INTER_AREA)
    
    return img

# 使用例：他の画像でテスト
def test_different_images():
    """異なる画像でパラメータをテスト"""
    
    # テスト画像のパス（存在する場合）
    test_images = [
        r"input\背景カットなし\sample\OK\OK2.png",
        r"input\背景カットなし\sample\削り節\削り節1.png",
        r"input\背景カットなし\sample\欠け\欠け1.png",
        r"input\背景カットなし\sample\汚れ\汚れ1.png"
    ]
    
    for img_path in test_images:
        if os.path.exists(img_path):
            print(f"\n=== テスト画像: {os.path.basename(img_path)} ===")
            
            # 元画像読み込み（日本語パス対応）
            original = read_image_japanese_path(img_path, cv2.IMREAD_GRAYSCALE)
            
            if original is not None:
                # 前処理適用
                processed = preprocess_image_with_params(img_path)
                
                if processed is not None:
                    # 比較表示
                    show_image_comparison(
                        original, 
                        processed, 
                        f"Preprocessing Result: {os.path.basename(img_path)}",
                        "Default Parameters"
                    )
                else:
                    print(f"前処理に失敗: {img_path}")
            else:
                print(f"元画像の読み込みに失敗: {img_path}")
        else:
            print(f"画像が見つかりません: {img_path}")

# 簡単なテスト用関数
def quick_test_image_loading():
    """画像読み込みのクイックテスト"""
    test_path = r"input\背景カットなし\sample\OK\OK1.png"
    
    print("=== 画像読み込みテスト ===")
    print(f"テストパス: {test_path}")
    print(f"ファイル存在: {os.path.exists(test_path)}")
    
    # 標準のcv2.imread
    img1 = cv2.imread(test_path, cv2.IMREAD_GRAYSCALE)
    print(f"標準cv2.imread: {'成功' if img1 is not None else '失敗'}")
    
    # 日本語パス対応版
    img2 = read_image_japanese_path(test_path, cv2.IMREAD_GRAYSCALE)
    print(f"日本語パス対応版: {'成功' if img2 is not None else '失敗'}")
    
    if img2 is not None:
        print(f"画像サイズ: {img2.shape}")
        print(f"データ型: {img2.dtype}")
        print(f"値の範囲: {img2.min()} - {img2.max()}")

# クイックテストを実行
quick_test_image_loading()

print("\n=== 異なる画像でのテスト ===")
test_different_images()

print("\n=== パラメータ調整のヒント ===")
print("• 暗い画像: gamma を小さく (0.5-1.0)")
print("• 明るい画像: gamma を大きく (1.5-2.0)")
print("• コントラストが低い: alpha を大きく (1.5-2.0)")
print("• ノイズが多い: denoise_h を大きく (50-100)")
print("• エッジが弱い: sharpen_strength を 'strong' に")
print("• 背景が多い: crop_ratio を小さく (0.5-0.6)")

print("\n=== 修正された機能 ===")
print("✓ 日本語パスに対応した画像読み込み")
print("✓ エラーハンドリングの改善")
print("✓ 画像読み込みテスト機能の追加")